In [31]:
# Cài thư viện nếu cần
!pip install pandas scikit-learn matplotlib seaborn --quiet


#Bước 1: Tải và đọc dữ liệu

In [32]:
import pandas as pd
from google.colab import drive

# 1. Gắn Google Drive của bạn
# Khi chạy đoạn này, bạn sẽ được yêu cầu cấp quyền truy cập.
# Hãy làm theo hướng dẫn trong output.
drive.mount('/content/gdrive')

# 2. Đọc dữ liệu từ Google Drive
# Thay đổi đường dẫn dưới đây cho phù hợp với vị trí file của bạn trong Drive.
# Ví dụ: nếu file 'annonimized.csv' của bạn nằm trong thư mục 'My Drive/data/'
# thì đường dẫn sẽ là '/content/gdrive/MyDrive/data/annonimized.csv'

# 'annonimized.csv'
df_annonimized = pd.read_csv("/content/gdrive/MyDrive/study/kì 8/Máy học/Đồ án cuối kỳ/annonimized.csv")




#  'th-public.csv'
df_qt_public = pd.read_csv("/content/gdrive/MyDrive/study/kì 8/Máy học/Đồ án cuối kỳ/qt-public.csv")







Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Bước 2: Tiền xử lý dữ liệu


In [33]:
import numpy as np

# Chuyển datetime
df['created_at'] = pd.to_datetime(df['created_at'], format='%m-%d %H:%M:%S', errors='coerce')
df['updated_at'] = pd.to_datetime(df['updated_at'], format='%m-%d %H:%M:%S', errors='coerce')


# Trích đặc trưng thời gian
df['submit_hour'] = df['created_at'].dt.hour
df['submit_weekday'] = df['created_at'].dt.weekday
df['delay'] = (df['updated_at'] - df['created_at']).dt.total_seconds()

# Use raw 'concat('it001', username)' as 'mssv'
df['mssv'] = df["concat('it001', username)"]

# Encode categorical
df['assignment_id'] = df["concat('it001',`assignment_id`)"].astype('category').cat.codes
df['problem_id'] = df["concat('it001',`problem_id`)"].astype('category').cat.codes

# Đảm bảo số liệu
df['percent_testcase_passed'] = df['pre_score'].fillna(0) / 100 # Assuming pre_score is out of 10000
df['delay_weight'] = df['coefficient'].fillna(100) / 100 # Assuming coefficient is out of 100
df['has_score'] = df['status'] == 'SCORE'
df['is_passed'] = df['percent_testcase_passed'] == 100

# Preview
df[['mssv', 'assignment_id', 'problem_id', 'percent_testcase_passed', 'submit_hour', 'submit_weekday', 'delay', 'delay_weight', 'has_score', 'is_passed']].head()

,mssv,assignment_id,problem_id,percent_testcase_passed,submit_hour,submit_weekday,delay,delay_weight,has_score,is_passed
0,ed9eaeb6a707f50154024b24d7efcb874a9795dd,116,208,0.0,8.0,1.0,294.0,1.0,True,False
1,ed9eaeb6a707f50154024b24d7efcb874a9795dd,116,208,0.0,8.0,1.0,10.0,1.0,True,False
2,ed9eaeb6a707f50154024b24d7efcb874a9795dd,116,208,100.0,8.0,1.0,9.0,1.0,True,True
3,ed9eaeb6a707f50154024b24d7efcb874a9795dd,116,335,100.0,8.0,1.0,9.0,1.0,True,True
4,ed9eaeb6a707f50154024b24d7efcb874a9795dd,116,212,100.0,9.0,1.0,10.0,1.0,True,True


#Bước 3: Trích đặc trưng cho từng sinh viên



In [34]:
# Gom theo mssv để tạo 1 dòng/mssv
features = df.groupby('mssv').agg({
    'percent_testcase_passed': ['mean', 'std', 'max'],
    'delay_weight': 'mean',
    'submit_hour': 'mean',
    'submit_weekday': 'nunique',
    'assignment_id': 'nunique',
    'problem_id': 'nunique',
    'delay': 'mean',
    'has_score': 'sum',
    'is_passed': 'sum'
})

# Đổi tên cột
features.columns = ['_'.join(col) for col in features.columns]
features.reset_index(inplace=True)

# Preview
features.head()

,mssv,percent_testcase_passed_mean,percent_testcase_passed_std,percent_testcase_passed_max,delay_weight_mean,submit_hour_mean,submit_weekday_nunique,assignment_id_nunique,problem_id_nunique,delay_mean,has_score_sum,is_passed_sum
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,55.041497,45.414358,100.0,1.000000,4.401361,7,7,46,8195.428571,119,54
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,54.885521,43.237609,100.0,1.000000,10.525097,7,9,78,-232304.548263,209,85
2,01122b3ef7e59b84189e65985305f575d6bdf83c,59.737538,41.699366,100.0,1.000000,10.964103,7,7,66,-103020.830769,175,59
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,59.527600,46.403021,100.0,1.000000,8.610000,7,4,47,1514.090000,76,52
4,013de369c439ab0ead8aa7da64423aa395a8be39,64.744486,44.096659,100.0,0.971963,8.598131,4,8,52,184.037383,94,59


#Bước 4: Chuẩn bị nhãn (label)

In [36]:
# Đọc điểm thực hành thật
df_score_th = pd.read_csv("/content/gdrive/MyDrive/study/kì 8/Máy học/Đồ án cuối kỳ/th-public.csv")  # file này có 2 cột: mssv, diem_thuc_hanh

# Đổi tên cột 'hash' thành 'mssv' trong df_score_th
df_score_th = df_score_th.rename(columns={'hash': 'mssv'})

# Ghép vào feature
df_merged_th = pd.merge(features, df_score_th, on='mssv')
df_merged_th.head()

,mssv,percent_testcase_passed_mean,percent_testcase_passed_std,percent_testcase_passed_max,delay_weight_mean,submit_hour_mean,submit_weekday_nunique,assignment_id_nunique,problem_id_nunique,delay_mean,has_score_sum,is_passed_sum,TH
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,55.041497,45.414358,100.0,1.000000,4.401361,7,7,46,8195.428571,119,54,5
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,54.885521,43.237609,100.0,1.000000,10.525097,7,9,78,-232304.548263,209,85,8.5
2,01122b3ef7e59b84189e65985305f575d6bdf83c,59.737538,41.699366,100.0,1.000000,10.964103,7,7,66,-103020.830769,175,59,7
3,013de369c439ab0ead8aa7da64423aa395a8be39,64.744486,44.096659,100.0,0.971963,8.598131,4,8,52,184.037383,94,59,10
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,52.400251,46.917106,100.0,1.000000,11.989950,7,9,90,-252938.427136,154,79,6


#Bước 5: Train/Test mô hình

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import numpy as np

# Tách features và labels
X = df_merged.drop(columns=['mssv', 'TH'])
y = df_merged['TH']

# Chia train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values with the mean of the training data
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_train.mean()) # Use mean from training data to avoid data leakage

# Convert y to numeric, coercing errors to NaN and then dropping NaN
y_train = pd.to_numeric(y_train, errors='coerce').dropna()
y_test = pd.to_numeric(y_test, errors='coerce').dropna()

# Ensure X_train and y_train have the same index after dropping NaNs in y_train
X_train = X_train.loc[y_train.index]
X_test = X_test.loc[y_test.index]


# Huấn luyện model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R^2 Score: {r2:.4f}")

TypeError: can only concatenate str (not "int") to str

#Bước 6: Dự đoán cho tất cả sinh viên

In [39]:
# Dự đoán cho tất cả sinh viên có trong annonimized.csv
features_all = df.groupby('mssv').agg({
    'percent_testcase_passed': ['mean', 'std', 'max'],
    'delay_weight': 'mean',
    'submit_hour': 'mean',
    'submit_weekday': 'nunique',
    'assignment_id': 'nunique',
    'problem_id': 'nunique',
    'delay': 'mean',
    'has_score': 'sum',
    'is_passed': 'sum'
})
features_all.columns = ['_'.join(col) for col in features_all.columns]
features_all.reset_index(inplace=True)

# Apply the same numeric conversion and imputation as the training data
for col in ['diemqt', 'CK', 'TH']: # Include all potential score columns
    if col in features_all.columns:
        features_all[col] = pd.to_numeric(features_all[col], errors='coerce')

# Impute missing values using the mean from the training data (assuming X_train_th mean is representative)
# This step is important to ensure consistency in feature scaling and handling missing values
# We need to use the mean calculated during the training phase (from X_train_th)

# Check if X_train_th_mean exists, if not, calculate it from X_train_th
if 'X_train_th_mean' not in locals():
    # If X_train_th was dropped due to previous error, recalculate it based on df_merged_th
    X_th = df_merged_th.drop(columns=['mssv', 'TH'])
    for col in ['diemqt', 'CK']:
        if col in X_th.columns:
            X_th[col] = pd.to_numeric(X_th[col], errors='coerce')
    y_th = pd.to_numeric(df_merged_th['TH'], errors='coerce').dropna()
    X_th = X_th.loc[y_th.index] # Ensure X_th and y_th have the same index
    X_train_th, X_test_th, y_train_th, y_test_th = train_test_split(X_th, y_th, test_size=0.2, random_state=42)
    X_train_th_mean = X_train_th.mean()


X_all = features_all.drop(columns=['mssv'])

# Align columns of X_all with X_train_th before imputation and prediction
X_all = X_all.reindex(columns=X_train_th.columns, fill_value=0) # Use reindex to align columns and fill new columns with 0 or a suitable value

X_all = X_all.fillna(X_train_th_mean)


# Dự đoán
y_all_pred_th = model_th.predict(X_all)

# Tạo file submission cho TH
submission_th = pd.DataFrame({
    'mssv': features_all['mssv'],
    'diem_du_doan_th': y_all_pred_th
})

# Làm tròn nếu muốn
submission_th['diem_du_doan_th'] = submission_th['diem_du_doan_th'].clip(0, 10).round(2)
submission_th.head()

,mssv,diem_du_doan_th
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,5.78
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,8.31
2,01122b3ef7e59b84189e65985305f575d6bdf83c,6.96
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,6.64
4,013de369c439ab0ead8aa7da64423aa395a8be39,9.02


#Bước 7: Xuất file để nộp

In [41]:
from google.colab import files

# Xuất file
submission_th.to_csv("submission_thuc_hanh.csv", index=False,header=False)

# Tải về
files.download("submission_thuc_hanh.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Bước 5: Train/Test mô hình cho điểm Quá trình (QT)

In [43]:
# Đọc điểm quá trình thật
df_score_qt = pd.read_csv("/content/gdrive/MyDrive/study/kì 8/Máy học/Đồ án cuối kỳ/qt-public.csv")

# Đổi tên cột 'hash' thành 'mssv'
df_score_qt = df_score_qt.rename(columns={'hash': 'mssv'})

# Ghép vào feature
df_merged_qt = pd.merge(features, df_score_qt, on='mssv')

# Tách features và labels
X_qt = df_merged_qt.drop(columns=['mssv', 'diemqt'])
y_qt = df_merged_qt['diemqt']

# Convert potentially non-numeric columns in X_qt to numeric, coercing errors to NaN
for col in ['TH', 'CK']: # Assuming these might be the problematic columns
    if col in X_qt.columns:
        X_qt[col] = pd.to_numeric(X_qt[col], errors='coerce')


# Chia train/test
X_train_qt, X_test_qt, y_train_qt, y_test_qt = train_test_split(X_qt, y_qt, test_size=0.2, random_state=42)

# Convert y_qt to numeric, coercing errors to NaN and then dropping NaN
y_train_qt = pd.to_numeric(y_train_qt, errors='coerce').dropna()
y_test_qt = pd.to_numeric(y_test_qt, errors='coerce').dropna()

# Ensure X_train_qt and y_train_qt have the same index after dropping NaNs in y_train_qt
X_train_qt = X_train_qt.loc[y_train_qt.index]
X_test_qt = X_test_qt.loc[y_test_qt.index]

# Impute missing values with the mean of the training data
X_train_qt = X_train_qt.fillna(X_train_qt.mean())
X_test_qt = X_test_qt.fillna(X_train_qt.mean()) # Use mean from training data to avoid data leakage


# Huấn luyện model
model_qt = RandomForestRegressor(n_estimators=100, random_state=42)
model_qt.fit(X_train_qt, y_train_qt)

# Dự đoán và đánh giá
y_pred_qt = model_qt.predict(X_test_qt)
r2_qt = r2_score(y_test_qt, y_pred_qt)
print(f"R^2 Score (QT): {r2_qt:.4f}")

R^2 Score (QT): 0.1062


# Bước 5: Train/Test mô hình cho điểm Cuối kỳ (CK)

In [44]:
# Đọc điểm cuối kỳ thật
df_score_ck = pd.read_csv("/content/gdrive/MyDrive/study/kì 8/Máy học/Đồ án cuối kỳ/ck-public.csv")

# Đổi tên cột 'hash' thành 'mssv'
df_score_ck = df_score_ck.rename(columns={'hash': 'mssv'})

# Ghép vào feature
df_merged_ck = pd.merge(features, df_score_ck, on='mssv')

# Tách features và labels
X_ck = df_merged_ck.drop(columns=['mssv', 'CK'])
y_ck = df_merged_ck['CK']

# Convert potentially non-numeric columns in X_ck to numeric, coercing errors to NaN
for col in ['TH', 'diemqt']: # Assuming these might be the problematic columns
    if col in X_ck.columns:
        X_ck[col] = pd.to_numeric(X_ck[col], errors='coerce')


# Chia train/test
X_train_ck, X_test_ck, y_train_ck, y_test_ck = train_test_split(X_ck, y_ck, test_size=0.2, random_state=42)

# Convert y_ck to numeric, coercing errors to NaN and then dropping NaN
y_train_ck = pd.to_numeric(y_train_ck, errors='coerce').dropna()
y_test_ck = pd.to_numeric(y_test_ck, errors='coerce').dropna()

# Ensure X_train_ck and y_train_ck have the same index after dropping NaNs in y_train_ck
X_train_ck = X_train_ck.loc[y_train_ck.index]
X_test_ck = X_test_ck.loc[y_test_ck.index]


# Impute missing values with the mean of the training data
X_train_ck = X_train_ck.fillna(X_train_ck.mean())
X_test_ck = X_test_ck.fillna(X_train_ck.mean()) # Use mean from training data to avoid data leakage


# Huấn luyện model
model_ck = RandomForestRegressor(n_estimators=100, random_state=42)
model_ck.fit(X_train_ck, y_train_ck)

# Dự đoán và đánh giá
y_pred_ck = model_ck.predict(X_test_ck)
r2_ck = r2_score(y_test_ck, y_pred_ck)
print(f"R^2 Score (CK): {r2_ck:.4f}")

R^2 Score (CK): 0.1635


# Bước 6 & 7: Dự đoán và Xuất file cho điểm QT và CK

In [47]:
# Dự đoán điểm QT cho tất cả sinh viên
# Use the existing features_all DataFrame
# Align columns of features_all with X_train_qt before imputation and prediction for QT
features_qt_predict = features_all.reindex(columns=X_train_qt.columns, fill_value=0)
features_qt_predict = features_qt_predict.fillna(X_train_qt.mean()) # Use mean from X_train_qt for imputation

y_all_pred_qt = model_qt.predict(features_qt_predict)

# Tạo file submission cho QT
submission_qt = pd.DataFrame({
    'mssv': features_all['mssv'],
    'diem_du_doan_qt': y_all_pred_qt
})

# Làm tròn và giới hạn điểm nếu muốn
submission_qt['diem_du_doan_qt'] = submission_qt['diem_du_doan_qt'].clip(0, 10).round(1)

# Dự đoán điểm CK cho tất cả sinh viên
# Align columns of features_all with X_train_ck before imputation and prediction for CK
features_ck_predict = features_all.reindex(columns=X_train_ck.columns, fill_value=0)
features_ck_predict = features_ck_predict.fillna(X_train_ck.mean()) # Use mean from X_train_ck for imputation

y_all_pred_ck = model_ck.predict(features_ck_predict)

# Tạo file submission cho CK
submission_ck = pd.DataFrame({
    'mssv': features_all['mssv'],
    'diem_du_doan_ck': y_all_pred_ck
})

# Làm tròn và giới hạn điểm nếu muốn
submission_ck['diem_du_doan_ck'] = submission_ck['diem_du_doan_ck'].clip(0, 10).round(1)

display(submission_qt.head())
display(submission_ck.head())

,mssv,diem_du_doan_qt
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,7.67
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,7.76
2,01122b3ef7e59b84189e65985305f575d6bdf83c,7.77
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,8.94
4,013de369c439ab0ead8aa7da64423aa395a8be39,9.63


,mssv,diem_du_doan_ck
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,5.6
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,6.1
2,01122b3ef7e59b84189e65985305f575d6bdf83c,5.4
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,5.0
4,013de369c439ab0ead8aa7da64423aa395a8be39,5.1


In [48]:
from google.colab import files

# Xuất file submission cho QT
submission_qt.to_csv("submission_qua_trinh.csv", index=False, header=False)

# Tải về file QT
files.download("submission_qua_trinh.csv")

# Xuất file submission cho CK
submission_ck.to_csv("submission_cuoi_ky.csv", index=False, header=False)

# Tải về file CK
files.download("submission_cuoi_ky.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>